# PART1 DATA EXPLORATION

## Imports

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import Counter
from pprint import pprint
from matplotlib import pyplot as plt
import seaborn as sns
from enum import Enum

## Load Data